In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# os.chdir('/content/drive/MyDrive/Data/')
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
from tensorflow import keras
import numpy as np
import pandas as pd
import numpy as np
import pickle
import pydot
import matplotlib.pyplot as plt

Found GPU at: /device:GPU:0


In [2]:
trainInput1 = np.load('trainInput1.npy')
trainInput2 = np.load('trainInput2.npy')
trainOutput = np.load('trainOutput.npy')

testInput1 = np.load('testInput1.npy')
testInput2 = np.load('testInput2.npy')
testOutput = np.load('testOutput.npy')

valInput1 = np.load('valInput1.npy')
valInput2 = np.load('valInput2.npy')
valOutput = np.load('valOutput.npy')

In [3]:
trainInput2.shape

(140, 250, 250)

In [4]:
def modelCNN():
    
    input1 = keras.Input(shape=(trainInput1.shape[1], trainInput1.shape[2], 1), batch_size=5)
    input2 = keras.Input(shape=(trainInput1.shape[1], trainInput1.shape[2], 1), batch_size=5)

    merged_input = keras.layers.concatenate([input1, input2])

    encoded = keras.layers.Conv2D(filters=120, kernel_size=(11, 11), strides=1, activation='relu')(merged_input)
    encoded = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)(encoded)

    encoded = keras.layers.Conv2D(filters=60, kernel_size=(7, 7), strides=1, activation='relu')(encoded)
    encoded = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)(encoded)

    encoded = keras.layers.Conv2D(filters=30, kernel_size=(3, 3), strides=2, activation='relu')(encoded)
    encoded = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)(encoded)

    # flatten = keras.layers.Reshape(target_shape=(5880, 1))(encoded)
    # merged = keras.layers.Concatenate(axis=1)([flatten, input2])

    dense = keras.layers.Flatten()(encoded)

    # dense1 = keras.layers.Dense(5880)(dense)

    reshape = keras.layers.Reshape(target_shape=(14, 14, 30))(dense)

    decoded = keras.layers.UpSampling2D(size=(2, 2))(reshape)
    decoded = keras.layers.Conv2DTranspose(filters=30, kernel_size=(3, 3), strides=2, activation='relu')(decoded)

    decoded = keras.layers.UpSampling2D(size=(2, 2))(decoded)
    decoded = keras.layers.Conv2DTranspose(filters=60, kernel_size=(7, 7), strides=1, activation='relu')(decoded)

    decoded = keras.layers.UpSampling2D(size=(2, 2))(decoded)
    decoded = keras.layers.Conv2DTranspose(filters=120, kernel_size=(11, 11), strides=1, activation='relu')(decoded)

    output = keras.layers.Conv2DTranspose(filters=1, kernel_size=(1, 1), strides=1)(decoded)

    model = keras.Model(inputs=[input1, input2], outputs=output, name = 'AutoEncoder')
    model.summary()
    tf.keras.utils.plot_model(model, 'Stress_AutoEncoder_CNN.png', show_shapes=True)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=opt, loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])

    return model


model = modelCNN()

Model: "AutoEncoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(5, 250, 250, 1)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(5, 250, 250, 1)]   0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (5, 250, 250, 2)     0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (5, 240, 240, 120)   29160       concatenate[0][0]      

In [ ]:
# checkpoint_path = "training_1/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

history = model.fit(x = [trainInput1,trainInput2],
                     y= trainOutput,batch_size=5,epochs=1000,
                     validation_data=([valInput1,valInput2],valOutput),
                     validation_batch_size=5)



Epoch 1/1000
28/28 [==============================] - 6s 229ms/step - loss: 37610.4219 - root_mean_squared_error: 193.9341 - val_loss: 38819.8125 - val_root_mean_squared_error: 197.0274
Epoch 2/1000
28/28 [==============================] - 6s 230ms/step - loss: 29421.1758 - root_mean_squared_error: 171.5260 - val_loss: 35678.1797 - val_root_mean_squared_error: 188.8867
Epoch 3/1000
28/28 [==============================] - 6s 231ms/step - loss: 26416.9902 - root_mean_squared_error: 162.5330 - val_loss: 28321.3691 - val_root_mean_squared_error: 168.2895
Epoch 4/1000
28/28 [==============================] - 6s 231ms/step - loss: 25117.8164 - root_mean_squared_error: 158.4860 - val_loss: 28466.3691 - val_root_mean_squared_error: 168.7198
Epoch 5/1000
28/28 [==============================] - 7s 232ms/step - loss: 26937.1504 - root_mean_squared_error: 164.1254 - val_loss: 28779.4414 - val_root_mean_squared_error: 169.6450
Epoch 6/1000
28/28 [==============================] - 7s 233ms/step - 

Epoch 42/1000
28/28 [==============================] - 7s 238ms/step - loss: 21073.5469 - root_mean_squared_error: 145.1673 - val_loss: 27580.5371 - val_root_mean_squared_error: 166.0739
Epoch 43/1000
28/28 [==============================] - 7s 238ms/step - loss: 20521.7949 - root_mean_squared_error: 143.2543 - val_loss: 24515.8223 - val_root_mean_squared_error: 156.5753
Epoch 44/1000
28/28 [==============================] - 7s 239ms/step - loss: 20296.3770 - root_mean_squared_error: 142.4653 - val_loss: 24272.9531 - val_root_mean_squared_error: 155.7978
Epoch 45/1000
28/28 [==============================] - 7s 238ms/step - loss: 20167.3496 - root_mean_squared_error: 142.0118 - val_loss: 25095.9629 - val_root_mean_squared_error: 158.4171
Epoch 46/1000
28/28 [==============================] - 7s 239ms/step - loss: 21028.8340 - root_mean_squared_error: 145.0132 - val_loss: 25009.7305 - val_root_mean_squared_error: 158.1447
Epoch 47/1000
28/28 [==============================] - 7s 239ms/s

Epoch 86/1000
28/28 [==============================] - 7s 241ms/step - loss: 18749.2305 - root_mean_squared_error: 136.9278 - val_loss: 24792.5547 - val_root_mean_squared_error: 157.4565
Epoch 87/1000
28/28 [==============================] - 7s 241ms/step - loss: 18933.0410 - root_mean_squared_error: 137.5974 - val_loss: 23891.7695 - val_root_mean_squared_error: 154.5696
Epoch 88/1000
28/28 [==============================] - 7s 241ms/step - loss: 19061.6230 - root_mean_squared_error: 138.0638 - val_loss: 24386.7812 - val_root_mean_squared_error: 156.1627
Epoch 89/1000
28/28 [==============================] - 7s 241ms/step - loss: 18161.8555 - root_mean_squared_error: 134.7659 - val_loss: 25111.9688 - val_root_mean_squared_error: 158.4676
Epoch 90/1000
28/28 [==============================] - 7s 241ms/step - loss: 18950.3164 - root_mean_squared_error: 137.6601 - val_loss: 27514.5625 - val_root_mean_squared_error: 165.8752
Epoch 91/1000
28/28 [==============================] - 7s 240ms/s

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.legend(['Train', 'Validation'])
plt.xlabel('No of Epochs')
plt.ylabel('RMSE')

plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Train', 'Validation'])
plt.xlabel('No of Epochs')
plt.ylabel('Loss')
plt.tight_layout()

In [ ]:
testInput1.shape

In [ ]:
testPredicted = model.predict(x=[testInput1,testInput2],batch_size=5)

In [ ]:
testPredicted.shape

In [ ]:
nSamples = 200
trainSamples = int( 0.7*nSamples)
testSamples = int(0.1*nSamples)
valSamples = int(0.2*nSamples)
refinement = 0.8
lenGrid = 200

In [ ]:
maskedIndexFile = open('maskedIndexTotal_suffled.pkl', 'rb')
maskedIndexTotal = pickle.load(maskedIndexFile)
maskedIndexFile.close()



In [ ]:
x = np.arange(0, lenGrid, refinement)
y = np.arange(0, lenGrid, refinement)

x, y = np.meshgrid(x, y)

for i in range(0, testPredicted.shape[0]):

  zPredict= testPredicted[i][:][:][:]


  zPredict = np.squeeze(zPredict, axis=2)

  maskedIndex = maskedIndexTotal[valSamples+trainSamples+i]

  zActual = testOutput[i][:][:]

  zPredict[maskedIndex[0][:], maskedIndex[1][:]] = np.nan
  zActual[maskedIndex[0][:], maskedIndex[1][:]] = np.nan

  plt.figure(figsize=(10,5))
  plt.subplot(121)
  plt.contourf(x, y, zPredict , cmap='rainbow')
  plt.title('Predicted Stress')
  plt.colorbar()
  plt.subplot(122)
  plt.contourf(x,y,zActual,  cmap='rainbow')
  plt.title('Actual Stress')
  plt.colorbar()

plt.tight_layout()

In [ ]:
model.save('Stress_Auotencoder_CNN')